In [3]:
%pylab

Using matplotlib backend: TkAgg
Populating the interactive namespace from numpy and matplotlib


In [2]:
import tkinter as tk
from tkinter import filedialog

from scipy import signal
from scipy.signal import find_peaks

# %matplotlib qt

In [4]:
%run functions
matplotlib.rcParams.update({'font.size': 25})

In [45]:
# Constantes de la physique
# ABSOLUMENT RECHARGER APRÈS AVOIR EXECUTÉ LES CASES D'IMPORT AU DESSUS

C_e = 1.602e-19        # Coulomb
kb = 1.38064852*1e-23  # Boltzman
m_Ca = 40.078*1.66054e-27 # masse Ca 40.078
m_GM = 1e6*1.66054e-27 # la masse de la GMol
eps0 = 8.854187*1e-12  # permittivité électrique du vide

r0 = 2.5e-3 # 2.5e-3   # rayon piège Ca+
d0 = 4e-3/2            # longueur piège Ca+

Omega = 2.0e6*2*pi # 2.047e6

bk = 4 # nombre de barreaux par groupe (2 -> 4-pole , 4 -> 8-pole ...)

mkappa = 0.23          # écrantage piège réel GiantMol

wzLC = (2*pi*100000)**2
kappa_simion = m_Ca*d0**2*wzLC/(2*C_e)
print('%s = %f' % ('$\kappa_{simion}$',kappa_simion) )

zeta = kappa_simion*r0**2/d0**2

$\kappa_{simion}$ = 0.328007


In [226]:
file_path = '/home/adrien/RemoteFS/Rivendel/Simulations/20220916' # 20220610
# file_path = '/home/adrien/RemoteFS/Rivendel/Simulations/20220512'
# file_path = '/home/adrien/RemoteFS/Rivendel/Simulations/20220510'

all_subdir = []
all_Temp = []
all_trjN = []
all_trj1 = []
all_xva_3DHarmo = []
all_xva_start = []

# selection of all directories where
# there are data files
for i,j in enumerate(os.walk(file_path)):
    if 'Try' in j[0]:
        all_subdir.append(j[0])
    # selection of all data files
    # interesting me
    for k,l in enumerate(j[2]):
        if 'TrjN' in l:
            all_trjN.append(j[0]+'/'+l)
        if 'Trj1' in l:
            all_trj1.append(j[0]+'/'+l)
            #print(l)
        if 'SimuType0' and '.dat' in l:
            all_Temp.append(j[0]+'/'+l)
        if 'xva_3D' in l:
            all_xva_3DHarmo.append(j[0]+'/'+l)
        if 'xva_start' in l:
            all_xva_start.append(j[0]+'/'+l)
        #print(j[0],'\n')
all_subdir.sort()
all_Temp.sort()
all_xva_3DHarmo.sort()
all_xva_start.sort()
all_trjN.sort()
# for i in range(len(all_subdir)):
#     print(all_subdir[i],'\n',all_Temp[i],'\n',all_trjN[i],'\n')

In [227]:
all_Temp

['/home/adrien/RemoteFS/Rivendel/Simulations/20220916/DC00_RF00/Try00/Temp_SimuType0_N00001_Vrf0010_Udc0.3346D+00V_nt1000.dat',
 '/home/adrien/RemoteFS/Rivendel/Simulations/20220916/DC10_RF10/Try00/Temp_SimuType0_N00001_Vrf0065_Udc0.1671D+02V_nt1000.dat']

In [228]:
for j,k in enumerate(all_subdir):
    print(all_Temp[j])
    str_N  = all_Temp[j].find('_N')
    cond_N = all_Temp[j][str_N+3:str_N+7]
    str_v  = all_subdir[j].find('DC')
    cond   = all_Temp[j][str_v:str_v+9]
    print(cond)
    outfile = 'Time_and_temp_1ion_RFHEAT_N'+ str(cond_N) + '_' + str(cond)
    
    data = loadtxt(all_Temp[j])
    time_s = data[:,0]
    T_CM = data[:,1:4]
    T_aux = data[:,4:]
    T_aux_avg = mean(T_aux,axis=1)
    r2_v2 = load_xv2rlim(all_trjN[j])
    rv_1ion = load_x_afterstart(all_trjN[j][:-5]+'1.bin')
    #r2    = r2_v2[0:3,:]
    #v2    = r2_v2[3:6,:]
    #rlim  = r2_v2[6:9,:]
    
    np.savez( outfile, time=time_s, temp=T_aux_avg, r2_v2_rlim=r2_v2, rv_1ion=rv_1ion) # dirname_hc=re.sub(r'\\',r'/',dirname_hc)
    print(outfile) # , r2_v2_rlim=r2_v2

/home/adrien/RemoteFS/Rivendel/Simulations/20220916/DC00_RF00/Try00/Temp_SimuType0_N00001_Vrf0010_Udc0.3346D+00V_nt1000.dat
DC00_RF00
[360000]
[360000]
Time_and_temp_1ion_RFHEAT_N0001_DC00_RF00
/home/adrien/RemoteFS/Rivendel/Simulations/20220916/DC10_RF10/Try00/Temp_SimuType0_N00001_Vrf0065_Udc0.1671D+02V_nt1000.dat
DC10_RF10
[360000]
[360000]
Time_and_temp_1ion_RFHEAT_N0001_DC10_RF10


In [251]:
# infile = '/home/adrien/Documents/Programmes/Python/THESE/RF_Temp_Fit/Time_and_temp_1ion_RFHEAT_N0001_DC10_RF10.npz'
infile = '/home/adrien/Documents/Programmes/Python/THESE/RF_Temp_Fit/Time_and_temp_1ion_RFHEAT_N0001_DC00_RF00.npz'
with numpy.load(infile) as data:
    a = data['time']
    b = data['temp']
    c = data['r2_v2_rlim']
    d = data['rv_1ion']

In [252]:
step_start = 1000
step_stop = len(d[0]) - 1000
dt = diff(a)[0]/1000
fs = 1/dt

time = linspace(min(a),max(a),len(d[0]))[step_start:step_stop]

y = d[0][step_start:step_stop]

figure('trajectory',clear=True)
plot(time*1e3,y)
xlabel('t [ms]')
grid()
tight_layout()

sp = np.fft.fft(y)
freq = np.fft.fftfreq(time.shape[-1],dt)*1e-3
figure('FFT trajectory',clear=True)
# plt.plot(freq, sp.real)
plt.plot(freq, abs(sp.real))

peaks, _ = find_peaks(abs(sp.real), distance = 10, height=0.0001)
plt.plot(freq[peaks], abs(sp.real[peaks]), "P")

xlabel('f [kHz]')

grid()
xlim(-1e1,2.8e3)
tight_layout()

In [254]:
freq[peaks]

array([   58.02319767,  1941.97640217,  2058.02279751,  3941.97600201,
        4058.02239735, -4058.02239735, -3941.97600201, -2058.02279751,
       -1941.97640217,   -58.02319767])

In [255]:
( 58.023-57.735)/ 58.023 * 100

0.4963548937490371

In [250]:
(376.150-369.504)/376.150 * 100

1.7668483317825228

In [256]:
369.504**2/376.150**2

0.9649752086671018

In [247]:
(freq[peaks][1] + freq[peaks][2])/2

1999.9995998397503